In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.python.org/simple


In [1]:
import mne
import warnings
import numpy as np


from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.exceptions import ConvergenceWarning
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import FastICA
from sklearn.neural_network import MLPClassifier

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.preprocessing import ICA

from utils.Pipeline import FourierTransform, Float64CSP, EEGStandardScaler, WaveletTransform
from utils.Preprocessing import load_and_preprocess_data


In [2]:
mne.set_log_level("CRITICAL")

# Suppress warnings from LogisticRegression and CSP
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.linear_model._logistic')
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='mne')

In [3]:
# Define subjects and runs for training and testing
train_subjects = [57, 22, 86]
train_tasks = {
    'Task 1': [3, 7, 11],
    'Task 2': [4, 8, 12],
    'Task 3': [5, 9, 13],
    'Task 4': [6, 10, 14]
}

test_subjects = [34]
test_tasks = {
    'Task 1': [3],
    'Task 2': [4],
    'Task 3': [5],
    'Task 4': [6]
}

In [4]:
#task_1 (open and close left or right fist)
train_runs = train_tasks['Task 1']
train_epochs = load_and_preprocess_data(train_subjects, train_runs)

X_train = train_epochs.get_data().astype('float64')
y_train = train_epochs.events[:, -1]

display(X_train.shape)
display(y_train.shape)


test_runs = test_tasks['Task 1']
test_epochs = load_and_preprocess_data(test_subjects, test_runs)

X_test = test_epochs.get_data().astype('float64')
y_test = test_epochs.events[:, -1]

display(X_test.shape)
display(y_test.shape)

(195, 64, 481)

(195,)

(22, 64, 481)

(22,)

In [5]:
# Aplanar los datos de entrenamiento
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Visualizar las nuevas formas
display(X_train_flattened.shape)
display(X_train.shape)

display(X_test_flattened.shape)


(195, 30784)

(195, 64, 481)

(22, 30784)

In [6]:
std_pca_dslr = Pipeline([
    ('standarize', StandardScaler()),
    ('pca', PCA(n_components=25)),
    ('dslr', LogisticRegression(max_iter=10000))
])

display(std_pca_dslr)

Pipeline(steps=[('standarize', StandardScaler()), ('pca', PCA(n_components=25)),
                ('dslr', LogisticRegression(max_iter=10000))])

In [7]:
param_grid = [
    # lbfgs y penalty l2
    {'pca__n_components': [5, 10, 15, 20],
     'dslr__solver': ['lbfgs'],
     'dslr__penalty': ['l2'],
     'dslr__C': [0.01, 0.1, 1, 10, 100]},
    
    # liblinear permite l1 y l2 penalties
    {'pca__n_components': [5, 10, 15, 20],
     'dslr__solver': ['liblinear'],
     'dslr__penalty': ['l1', 'l2'],
     'dslr__C': [0.01, 0.1, 1, 10, 100]}
]

In [8]:
# SIN GRIND SEARCH
# Entrenar el modelo
std_pca_dslr.fit(X_train_flattened, y_train)

# Evaluar el modelo en el conjunto de prueba
train_accuracy = std_pca_dslr.score(X_train_flattened, y_train)
test_accuracy = std_pca_dslr.score(X_test_flattened, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(std_pca_dslr, X_train_flattened, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 0.75
Precisión en prueba: 0.64
Precisión en cada pliegue: [0.64102564 0.69230769 0.56410256 0.69230769 0.64102564]
Precisión media: 0.65


In [9]:
# GRID SEARCH CV

""" # Crear el objeto GridSearchCV
grid_search = GridSearchCV(std_pca_dslr, param_grid, cv=5, n_jobs=-1)  # n_jobs=-1 usa todos los cores disponibles

# Entrenar con el conjunto de datos de entrenamiento (GridSearch)
grid_search.fit(X_train_flattened, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Evaluar el rendimiento del mejor modelo en los datos de prueba
test_accuracy = grid_search.score(X_test_flattened, y_test)
print(f"Precisión en prueba con mejores parámetros: {test_accuracy:.2f}")

# También puedes obtener la precisión media de los pliegues de validación cruzada
cv_results = grid_search.cv_results_
mean_test_score = np.mean(cv_results['mean_test_score'])
print(f"Precisión media de validación cruzada: {mean_test_score:.2f}") """

' # Crear el objeto GridSearchCV\ngrid_search = GridSearchCV(std_pca_dslr, param_grid, cv=5, n_jobs=-1)  # n_jobs=-1 usa todos los cores disponibles\n\n# Entrenar con el conjunto de datos de entrenamiento (GridSearch)\ngrid_search.fit(X_train_flattened, y_train)\n\n# Mostrar los mejores parámetros encontrados\nprint("Mejores parámetros encontrados:", grid_search.best_params_)\n\n# Evaluar el rendimiento del mejor modelo en los datos de prueba\ntest_accuracy = grid_search.score(X_test_flattened, y_test)\nprint(f"Precisión en prueba con mejores parámetros: {test_accuracy:.2f}")\n\n# También puedes obtener la precisión media de los pliegues de validación cruzada\ncv_results = grid_search.cv_results_\nmean_test_score = np.mean(cv_results[\'mean_test_score\'])\nprint(f"Precisión media de validación cruzada: {mean_test_score:.2f}") '

-----

In [11]:
fft_csp_svm = Pipeline([
    ('fft', FourierTransform()), 
    ('csp', CSP(n_components=4, reg=None, log=True)),  # Common Spatial Patterns
    ('svm', SVC(kernel='linear', C=1.0))  # Clasificador SVM con kernel lineal
])

# Definir los hiperparámetros a optimizar
param_grid = {
    'csp__n_components': [2, 4, 6, 8],  # Probar con diferentes números de componentes de CSP
    'svm__C': [0.1, 1, 10, 100],        # Parámetro de regularización de SVM
}

In [12]:
# Entrenar el modelo
fft_csp_svm.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
train_accuracy = fft_csp_svm.score(X_train, y_train)
test_accuracy = fft_csp_svm.score(X_test, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(fft_csp_svm, X_train, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 0.76
Precisión en prueba: 0.64
Precisión en cada pliegue: [0.51282051 0.64102564 0.46153846 0.64102564 0.51282051]
Precisión media: 0.55


In [13]:
""" # GRID SEARCH CV

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(fft_csp_svm, param_grid, cv=5, n_jobs=-1)  # n_jobs=-1 usa todos los cores disponibles

# Entrenar con el conjunto de datos de entrenamiento (GridSearch)
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Evaluar el rendimiento del mejor modelo en los datos de prueba
test_accuracy = grid_search.score(X_test, y_test)
print(f"Precisión en prueba con mejores parámetros: {test_accuracy:.2f}")

# También puedes obtener la precisión media de los pliegues de validación cruzada
cv_results = grid_search.cv_results_
mean_test_score = np.mean(cv_results['mean_test_score'])
print(f"Precisión media de validación cruzada: {mean_test_score:.2f}") """

' # GRID SEARCH CV\n\n# Crear el objeto GridSearchCV\ngrid_search = GridSearchCV(fft_csp_svm, param_grid, cv=5, n_jobs=-1)  # n_jobs=-1 usa todos los cores disponibles\n\n# Entrenar con el conjunto de datos de entrenamiento (GridSearch)\ngrid_search.fit(X_train, y_train)\n\n# Mostrar los mejores parámetros encontrados\nprint("Mejores parámetros encontrados:", grid_search.best_params_)\n\n# Evaluar el rendimiento del mejor modelo en los datos de prueba\ntest_accuracy = grid_search.score(X_test, y_test)\nprint(f"Precisión en prueba con mejores parámetros: {test_accuracy:.2f}")\n\n# También puedes obtener la precisión media de los pliegues de validación cruzada\ncv_results = grid_search.cv_results_\nmean_test_score = np.mean(cv_results[\'mean_test_score\'])\nprint(f"Precisión media de validación cruzada: {mean_test_score:.2f}") '

------

In [14]:
fourier_csp_lda = Pipeline([
        ('fourier', FourierTransform()),                 # Apply Fourier Transform to the data
        ('scaler', EEGStandardScaler()),                 # Standardise the data
        ('CSP', Float64CSP(n_components=4, reg=None, log=True)),  # Apply Common Spatial Patterns (CSP)
        ('LDA', LinearDiscriminantAnalysis())            # Use Linear Discriminant Analysis for classification
    ])

In [15]:
# SIN GRIND SEARCH
# Entrenar el modelo
fourier_csp_lda.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
train_accuracy = fourier_csp_lda.score(X_train, y_train)
test_accuracy = fourier_csp_lda.score(X_test, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(fourier_csp_lda, X_train, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 0.66
Precisión en prueba: 0.64
Precisión en cada pliegue: [0.64102564 0.61538462 0.61538462 0.66666667 0.66666667]
Precisión media: 0.64


In [16]:
fourier_csp_svm = Pipeline([
        ('fourier', FourierTransform()),                 # Apply Fourier Transform to the data
        ('scaler', EEGStandardScaler()),                 # Standardise the data
        ('CSP', Float64CSP(n_components=4, reg=None, log=True)),  # Apply Common Spatial Patterns (CSP)
        ('SVM', SVC(kernel='linear'))                    # Use Support Vector Machine with a linear kernel
    ])

In [17]:
# SIN GRIND SEARCH
# Entrenar el modelo
fourier_csp_svm.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
train_accuracy = fourier_csp_svm.score(X_train, y_train)
test_accuracy = fourier_csp_svm.score(X_test, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(fourier_csp_svm, X_train, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 0.65
Precisión en prueba: 0.64
Precisión en cada pliegue: [0.64102564 0.64102564 0.64102564 0.64102564 0.66666667]
Precisión media: 0.65


---

In [18]:
wavelet_csp_rf = Pipeline([
    ('wavelet', WaveletTransform(wavelet='db4', level=4)),  # Transformada Wavelet Discreta
    ('rf', RandomForestClassifier(n_estimators=100))        # Random Forest Classifier
])

# Definir los hiperparámetros a optimizar
param_grid = {
    'csp__n_components': [2, 4, 6, 8],  # Probar con diferentes números de componentes de CSP
    'rf__n_estimators': [50, 100, 200],  # Número de árboles en Random Forest
    'rf__max_depth': [None, 10, 20, 30]  # Profundidad máxima de los árboles
}
wavelet_csp_rf.fit(X_train, y_train)
# Evaluar el modelo en el conjunto de prueba
train_accuracy = wavelet_csp_rf.score(X_train, y_train)
test_accuracy = wavelet_csp_rf.score(X_test, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(wavelet_csp_rf, X_train, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 1.00
Precisión en prueba: 0.64
Precisión en cada pliegue: [0.66666667 0.76923077 0.71794872 0.66666667 0.69230769]
Precisión media: 0.70


In [19]:
csp_rf = Pipeline([
    ('csp', Float64CSP(n_components=4, reg=None, log=True)),  # Transformada Wavelet Discreta
    ('rf', RandomForestClassifier(n_estimators=100))        # Random Forest Classifier
])


csp_rf.fit(X_train, y_train)
# Evaluar el modelo en el conjunto de prueba
train_accuracy = csp_rf.score(X_train, y_train)
test_accuracy = csp_rf.score(X_test, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(csp_rf, X_train, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 1.00
Precisión en prueba: 0.68
Precisión en cada pliegue: [0.64102564 0.53846154 0.51282051 0.53846154 0.61538462]
Precisión media: 0.57


In [20]:
ft_rf = Pipeline([
    ('ft', FourierTransform()),  # Transformada Wavelet Discreta
    ('rf', RandomForestClassifier(n_estimators=100))        # Random Forest Classifier
])


ft_rf.fit(X_train_flattened, y_train)
# Evaluar el modelo en el conjunto de prueba
train_accuracy = ft_rf.score(X_train_flattened, y_train)
test_accuracy = ft_rf.score(X_test_flattened, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(ft_rf, X_train_flattened, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 1.00
Precisión en prueba: 0.36
Precisión en cada pliegue: [0.64102564 0.58974359 0.64102564 0.66666667 0.69230769]
Precisión media: 0.65


In [21]:
wavelet_lr = Pipeline([
    ('wavelet', WaveletTransform(wavelet='db4', level=4)),  # Transformada Wavelet Discreta
    ('dslr', LogisticRegression(max_iter=10000))        # Random Forest Classifier
])

wavelet_csp_rf.fit(X_train, y_train)
# Evaluar el modelo en el conjunto de prueba
train_accuracy = wavelet_csp_rf.score(X_train, y_train)
test_accuracy = wavelet_csp_rf.score(X_test, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(wavelet_csp_rf, X_train, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 1.00
Precisión en prueba: 0.64
Precisión en cada pliegue: [0.66666667 0.71794872 0.69230769 0.69230769 0.66666667]
Precisión media: 0.69


In [22]:
wavelet_lr = Pipeline([
    ('wavelet', WaveletTransform(wavelet='db4', level=4)),  # Transformada Wavelet Discreta
    ('svm', SVC(kernel='linear'))        # Random Forest Classifier
])

wavelet_lr.fit(X_train, y_train)
# Evaluar el modelo en el conjunto de prueba
train_accuracy = wavelet_lr.score(X_train, y_train)
test_accuracy = wavelet_lr.score(X_test, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(wavelet_lr, X_train, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 0.65
Precisión en prueba: 0.64
Precisión en cada pliegue: [0.64102564 0.64102564 0.64102564 0.64102564 0.66666667]
Precisión media: 0.65


In [7]:
mlp = Pipeline([
    #('scaler', StandardScaler()),               # Paso 2: Estandarización
    #('ica', FastICA(n_components=20, random_state=42)),  # Paso 3: Análisis de Componentes Independientes
    #('pca', PCA(n_components=10)),              # Paso 4: Reducción de dimensionalidad con PCA
    ('mlp', MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500))  # Paso 5: Red Neuronal
])

mlp.fit(X_train_flattened, y_train)
# Evaluar el modelo en el conjunto de prueba
train_accuracy = mlp.score(X_train_flattened, y_train)
test_accuracy = mlp.score(X_test_flattened, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(mlp, X_train_flattened, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 1.00
Precisión en prueba: 0.64
